In [1]:
import datetime
import logging
import time
from pathlib import Path

In [2]:
import pandas as pd

In [3]:
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.utils.export import generate_multimodal_pages
from docling.utils.utils import create_hash

/opt/anaconda3/envs/damg7245/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
_log = logging.getLogger(__name__)

In [5]:
IMAGE_RESOLUTION_SCALE = 2.0

In [12]:
def main():
    logging.basicConfig(level=logging.INFO)

    input_doc_path = Path("Samples/wikitest1.pdf")
    output_dir = Path("Samples")

    # Important: For operating with page images, we must keep them, otherwise the DocumentConverter
    # will destroy them for cleaning up memory.
    # This is done by setting AssembleOptions.images_scale, which also defines the scale of images.
    # scale=1 correspond of a standard 72 DPI image
    pipeline_options = PdfPipelineOptions()
    pipeline_options.images_scale = IMAGE_RESOLUTION_SCALE
    pipeline_options.generate_page_images = True

    doc_converter = DocumentConverter(
        format_options={
            InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
        }
    )

    start_time = time.time()

    conv_res = doc_converter.convert(input_doc_path)

    output_dir.mkdir(parents=True, exist_ok=True)

    rows = []
    for (
        content_text,
        content_md,
        content_dt,
        page_cells,
        page_segments,
        page,
    ) in generate_multimodal_pages(conv_res):

        dpi = page._default_image_scale * 72

        rows.append(
            {
                "document": conv_res.input.file.name,
                "hash": conv_res.input.document_hash,
                "page_hash": create_hash(
                    conv_res.input.document_hash + ":" + str(page.page_no - 1)
                ),
                "image": {
                    "width": page.image.width,
                    "height": page.image.height,
                    "bytes": page.image.tobytes(),
                },
                "cells": page_cells,
                "contents": content_text,
                "contents_md": content_md,
                "contents_dt": content_dt,
                "segments": page_segments,
                "extra": {
                    "page_num": page.page_no + 1,
                    "width_in_points": page.size.width,
                    "height_in_points": page.size.height,
                    "dpi": dpi,
                },
            }
        )

    # Generate one parquet from all documents
    df = pd.json_normalize(rows)
    now = datetime.datetime.now()
    output_filename = output_dir / f"multimodal_{now:%Y-%m-%d_%H%M%S}.parquet"
    print(df)
    df.to_markdown()
    df.to_parquet(output_filename)

    end_time = time.time() - start_time

    _log.info(
        f"Document converted and multimodal pages generated in {end_time:.2f} seconds."
    )

    # This block demonstrates how the file can be opened with the HF datasets library
    from datasets import Dataset
    from PIL import Image
    multimodal_df = pd.read_parquet(output_filename)
    multimodal_df.head()

    # # Convert pandas DataFrame to Hugging Face Dataset and load bytes into image
    # dataset = Dataset.from_pandas(multimodal_df)
    # def transforms(examples):
    #     examples["image"] = Image.frombytes('RGB', (examples["image.width"], examples["image.height"]), examples["image.bytes"], 'raw')
    #     return examples
    # dataset = dataset.map(transforms)

In [13]:
if __name__ == "__main__":
    main()

INFO:docling.document_converter:Going to convert document batch...
INFO:docling.utils.accelerator_utils:Accelerator device: 'mps'
INFO:docling.utils.accelerator_utils:Accelerator device: 'mps'
INFO:docling.utils.accelerator_utils:Accelerator device: 'mps'
INFO:docling.pipeline.base_pipeline:Processing document wikitest1.pdf
INFO:docling.document_converter:Finished converting document wikitest1.pdf in 69.43 sec.


         document                                               hash  \
0   wikitest1.pdf  ea3bf30568489edae008ff5786f7d4c11cc512c811e337...   
1   wikitest1.pdf  ea3bf30568489edae008ff5786f7d4c11cc512c811e337...   
2   wikitest1.pdf  ea3bf30568489edae008ff5786f7d4c11cc512c811e337...   
3   wikitest1.pdf  ea3bf30568489edae008ff5786f7d4c11cc512c811e337...   
4   wikitest1.pdf  ea3bf30568489edae008ff5786f7d4c11cc512c811e337...   
5   wikitest1.pdf  ea3bf30568489edae008ff5786f7d4c11cc512c811e337...   
6   wikitest1.pdf  ea3bf30568489edae008ff5786f7d4c11cc512c811e337...   
7   wikitest1.pdf  ea3bf30568489edae008ff5786f7d4c11cc512c811e337...   
8   wikitest1.pdf  ea3bf30568489edae008ff5786f7d4c11cc512c811e337...   
9   wikitest1.pdf  ea3bf30568489edae008ff5786f7d4c11cc512c811e337...   
10  wikitest1.pdf  ea3bf30568489edae008ff5786f7d4c11cc512c811e337...   

                                            page_hash  \
0   d6363da410be8d04f87aae6a6733395ded234f1e8ba7da...   
1   0ec4785d564c845c0

INFO:__main__:Document converted and multimodal pages generated in 70.35 seconds.
